In [ ]:
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
with open('../../spbu-ai-fundamentals/config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

В этой теме мы поработаем с данными, посвященными определению рака молочной железы на основе различных признаков анализа клеток в биопсии (радиус, кривизна, симметрия). Известно, что этот датасет линейно разделим.

In [ ]:
df = pd.read_csv("D:\\spbu-ai-fundamentals\\practicum_7\\wdbc\\data.csv")
df.head()

In [ ]:
df.info()

**Задание**: Проведите краткий EDA. Есть ли выбросы в данных, какие столбцы коррелируют больше всего, стоит ли преобразоывавть какие-то признаки? Хватит 3-4 графиков или таблиц (но можно больше).

In [ ]:
num_df = df.select_dtypes(exclude='object')
del num_df['id']
del num_df['Unnamed: 32']

In [ ]:
fig, axes = plt.subplots(6,5, figsize=(20,20))
axes_flattened = axes.flatten()

for i, col in enumerate(list(num_df.columns)):
    data_col = df[col].dropna()
    if data_col.nunique() <= 1:
        continue
    sns.boxplot(x='diagnosis', y=col, data=df, ax=axes_flattened[i])
    axes_flattened[i].set_title(f'{col}', fontsize=10)
    i += 1
plt.tight_layout(pad=2.0)

_Бокс-плоты позволяют увидеть, что очень много признаков значительно отличаются по среднему значению для зколачественных и доброкачественных опухолей. Так что предположительно на тепловой карте будут сильные корреляции. Еще видим, что выбросы действительно есть. А еще: у признаков очень отличается масштаб, то есть они несбалансированы._

In [ ]:
fig, axes = plt.subplots(6,5, figsize=(20,20))
axes_flattened = axes.flatten()
i = 0

for col in num_df.columns:
    data_col = df[col].dropna()
    if data_col.nunique() <= 1:
        continue
    sns.violinplot(x='diagnosis', y=col, data=df, ax=axes_flattened[i])
    axes_flattened[i].set_title(f'{col}', fontsize=10)
    i+=1
plt.tight_layout(pad=2.0)

_Это (на мой вкус) более приятный вид бокс-плотов._

In [ ]:
def corrplot(df, method="pearson", annot=True, **kwargs):
    sns.clustermap(
        df.corr(method),
        fmt=".2f", 
        cmap='coolwarm',
        vmin=-1.0,
        vmax=1.0,
        method="complete",
        annot=annot,
        figsize=(20,20),
        **kwargs,
    )
corrplot(num_df)

_Здесь видно: очень много значений гиперкоррелируют. Практически все признаки "связаны между собой"._

In [ ]:
fig, axes = plt.subplots(6,5, figsize=(20,20))
axes_flattened = axes.flatten()
i = 0

for col in num_df.columns:
    data_col = df[col].dropna()
    if data_col.nunique() <= 1:
        continue
    for d in df['diagnosis'].unique():
        sns.kdeplot(x=df[df['diagnosis'] == d][col], label=d, ax=axes_flattened[i])
    axes_flattened[i].set_title(f'{col}', fontsize=10)
    axes_flattened[i].legend()
    i+=1
plt.tight_layout(pad=2.0)

_Это очень информативный график, он показывает относительную плотность распределения значений признаков. Сразу видны отличия между категориями целевой переменной._

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df[num_df.columns].dropna(axis=1)
X_scaled = StandardScaler().fit_transform(X)
pca = PCA(n_components=2)
components = pca.fit_transform(X_scaled)

df_pca = pd.DataFrame(components, columns=['PC1', 'PC2'])
df_pca['diagnosis'] = df['diagnosis'].values

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='diagnosis')
plt.title('PCA: 2D пространство')

_Здесь методом главных компонент понизили размерность до двух признаков и видно, что диагнозы действительно хорошо отделимы._

In [ ]:
df = df.drop(['id', 'Unnamed: 32'], axis=1)
df.head()

In [ ]:
df['diagnosis'] = df['diagnosis'].replace({'B': 0, 'M': 1}).astype(int)
df.head()

**Задание**: выведите, сколько в датасете примеров позитивного и негативного класса.

In [ ]:
print(df['diagnosis'].value_counts())

In [ ]:
target = 'diagnosis'
features = list(df.columns)
features.remove('diagnosis')
features

In [ ]:
X = df[features]
y = df[[target]]

Попробуем обучить логистическую регрессию на этих данных. Обратите внимание, что по умолчанию применяется L2 регуляризация,мы будем строить предсказания без нее. Однако, в качестве упражнения, сравним результаты с масштабированием признаков и без.

**Задание**: оцените, насколько сбалансированы признаки по масштабу. Попробуйте ответить до запуска кода, стоит ли их сначала масштабировать и почему. 

In [ ]:
summary = pd.DataFrame({
    'mean': num_df.mean(),
    'min': num_df.min(),
    'max': num_df.max(),
    'range': num_df.max() - num_df.min(),
})

summary_sorted = summary.sort_values('range', ascending=False)
print("Признаки с наибольшим разбросом:")
display(summary_sorted[['range']].head())

print("Минимальный range:", summary['range'].min())
print("Максимальный range:", summary['range'].max())
print("Отношение max_range / min_range:", summary['range'].max() / summary['range'].min())

_Масштабирование 100% нужно, значения некоторых признакв отличаются на порядки._

Без масштабирования:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values.reshape(-1), train_size=0.8, shuffle=True)
clf = LogisticRegression(penalty=None)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

С масштабированием:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values.reshape(-1), train_size=0.8, shuffle=True)
clf = LogisticRegression(penalty=None)
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

Все классификаторы в Sklearn имеют два режима - предсказание лейблов и вероятностей. Предсказание вероятностей дает нам необработанные оценки принадлежности к тому или иному классу. Модель в таком случае возвращает вектор (для каждого семпла) размера N (где N - число классов). 

**Вопрос**: Какого размера будет предсказание в случае бинарной логистической регрессии? А многоклассовой? Другими словами, в каких случаях негативный класс добавляется как отдельный?

_В случае бинарной логистической регрессии predict_proba(X) вернёт массив с формой (n_samples, 2), а в случае многоклассовой — (n_samples, n_classes).
Негативный класс вроде как всегда включается явно._

In [ ]:
df_results = pd.DataFrame({
    'pred': clf.predict(X_test).reshape(-1),
    'pred_proba': clf.predict_proba(X_test)[:, 1],
    'true': y_test.reshape(-1),
})

**Задание**: Постройте матрицу предсказаний 100x2 для регрессии с двумя классами, где в каждой строке будут случайные значения. 
1) Получите из этого оценку принадлежности к классу с помощью сигмоиды и софтмакса. 
2) Постройте предсказание класса. В случае сигмоиды предсказывайте принадлежность к классу на основе границы, софтмакса - по максимальной вероятности

**Вопрос***: как еще можно предсказать класс? Всегда ли нужно брать именно эти функции?

In [ ]:
from scipy.special import expit, softmax
np.random.seed(42)
logits = np.random.randn(100, 2)

sigmoid_probs = expit(logits[:, 1])  # P(class=1)
softmax_probs = softmax(logits, axis=1)  # P(class=0), P(class=1)

sigmoid_preds = (sigmoid_probs >= 0.5).astype(int)
softmax_preds = np.argmax(softmax_probs, axis=1)

df_result = pd.DataFrame({
    'logit_0': logits[:, 0],
    'logit_1': logits[:, 1],
    'sigmoid_prob': sigmoid_probs,
    'softmax_prob_0': softmax_probs[:, 0],
    'softmax_prob_1': softmax_probs[:, 1],
    'sigmoid_pred': sigmoid_preds,
    'softmax_pred': softmax_preds
})

print(df_result.head())

In [ ]:
df_results.head(20)

# Метрики классификации


## Метрики на основе лейблов
Рассмотрим, какие у нас могут быть тезультаты классификации.

* TP (true positive) - правильно предсказали: рак есть, что модель и предсказала
* FP (false positive) - неправильно предсказали: рака нет,  а модель предсказала, что есть (1st order error)
* FN (false negative) - неправильно предсказали: рак вообще-то есть,  а модель предсказала, что нет (2nd order error)!
* TN (true negative) - правильно предсказали: рака нет, что модель и предсказала


Pos/Neg - общее количество объектов класса 1/0

Метрики:

* $ \text{Accuracy} = \frac{TP + TN}{Pos+Neg}$ - Доля правильных ответов
* $ \text{Error rate} = 1 -\text{accuracy}$ - Доля ошибок
* $ \text{Precision} =\frac{TP}{TP + FP}$ - Точность
* $ \text{Recall} =\frac{TP}{TP + FN} = \frac{TP}{Pos}$ - Полнота
* $ \text{F}_\beta \text{-score} = (1 + \beta^2) \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{(\beta^2 \cdot \mathrm{precision}) + \mathrm{recall}}$ F-мера (часто используют F1-меру, где $\beta=1$)

### ROC кривая

ROC кривая измеряет насколько хорошо классификатор разделяет два класса. Она построена на предсказании вероятности. Площадь под ней (ROC-AUC) является неплохой оценкой общего качества предсказаний. 
 
Пусть $y_{\rm i}$ - истинная метрка и $\hat{y}_{\rm i}$ - прогноз вероятности для $i^{\rm th}$ объекта.

Число положительных и отрицательных объектов: $\mathcal{I}_{\rm 1} = \{i: y_{\rm i}=1\}$ and $\mathcal{I}_{\rm 0} = \{i: y_{\rm i}=0\}$.

Для каждого порогового значения вероятности $\tau$ считаем True Positive Rate (TPR) и False Positive Rate (FPR):

\begin{equation}
TPR(\tau) = \frac{1}{I_{\rm 1}} \sum_{i \in \mathcal{I}_{\rm 1}} I[\hat{y}_{\rm i} \ge \tau] = \frac{TP(\tau)}{TP(\tau) + FN(\tau)} = \frac{TP(\tau)}{Pos}
\end{equation}

\begin{equation}
FPR(\tau) = \frac{1}{I_{\rm 0}} \sum_{i \in \mathcal{I}_{\rm 0}} I[\hat{y}_{\rm i} \ge \tau]= \frac{FP(\tau)}{FP(\tau) + TN(\tau)} = \frac{FP(\tau)}{Neg}
\end{equation}

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_samples=10000, n_features=10, n_informative=5, n_redundant=5, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Используем для сравнения случайный предикт. Иногда это не худшая стратегия. Если в данных мало сигнала, случайное предсказание может работать лучше ложного.

In [ ]:
from sklearn.dummy import DummyClassifier
random_classifier = DummyClassifier(strategy='uniform', random_state=42).fit(X_train, y_train)
y_random = random_classifier.predict_proba(X_test)[:,1]
y_random

In [ ]:
random_preds = random_classifier.predict(X_test)
random_preds

Построим 

In [ ]:
from sklearn.metrics import average_precision_score

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay

def depict_pr_roc(y_true, y_pred, classifier_name='Some Classifier', ax=None):
    if ax is None:
        fig, ax = plt.subplots(1, 2, figsize=(11, 5))

    print(classifier_name, 'metrics')
    PrecisionRecallDisplay.from_predictions(y_true, y_pred, ax=ax[0], name=classifier_name)
    print('AUC-PR: %.4f' % average_precision_score(y_true, y_pred))
    ax[0].set_title("PRC")
    ax[0].set_ylim(0, 1.1)

    RocCurveDisplay.from_predictions(y_true, y_pred, ax=ax[1], name=classifier_name)
    print('AUC-ROC: %.4f' % roc_auc_score(y_true, y_pred))
    ax[1].set_title("ROC")
    ax[1].set_ylim(0, 1.1)

    plt.tight_layout()
    plt.legend()


depict_pr_roc(y_test, y_random, 'Random Classifier')

Также посчитаем другие метрики на основе лейблов.

**Задание:** Дополните код по рассчету метрик.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def quality_metrics_report(y_true, y_pred):

    tp = np.sum( (y_true == 1) * (y_pred == 1) )
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    tn = np.sum( (y_true == 0) * (y_pred == 0) )

    accuracy = accuracy_score(y_true, y_pred)
    error_rate = 1 - accuracy
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return [tp, fp, fn, tn, accuracy, error_rate, precision, recall, f1]

In [ ]:
# dataframe для сравнения
# методов классификации по метрикам
df_metrics = pd.DataFrame(
    columns=['acc', 'er', 'precision', 'recall', 'f1', 'auc_pr', 'roc_auc_score', 'reg_const']
)
precision, recall, _ = precision_recall_curve(y_test, y_random)
# добавление очередной строки с характеристиками метода
[tp, fp, fn, tn, accuracy, error_rate, precision, recall, f1] = quality_metrics_report(y_test, random_preds)
df_metrics.loc['Random Classifier'] = [
      accuracy, error_rate, precision, recall, f1,
      average_precision_score(y_test, y_random),
      roc_auc_score(y_test, y_random),
      0,
]

# по аналогии результаты следующих экспериментов можно будет собрать в табличку
df_metrics

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

y_lr_prob = lr.predict_proba(X_test)[:, 1] 
y_lr_pred = lr.predict(X_test) 

[tp, fp, fn, tn, accuracy, error_rate, precision_val, recall_val, f1_val] = quality_metrics_report(y_test, y_lr_pred)

df_metrics.loc['Logistic Regression'] = [
    accuracy,
    error_rate,
    precision_val,
    recall_val,
    f1_val,
    average_precision_score(y_test, y_lr_prob),
    roc_auc_score(y_test, y_lr_prob),
    0
]

df_metrics

Согласуются ли метрики? В чем может быть проблема accuracy?

**Задание**: Соберите табличку для разных классификаторов.

**Задание**: Постройте график PR-curve, ROC-curve для лучшего из них

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# PR-кривая
PrecisionRecallDisplay.from_predictions(y_test, y_lr_prob, ax=ax[0], name='Logistic Regression')
ax[0].set_title("Precision-Recall Curve")
ax[0].set_ylim(0, 1.05)

# ROC-кривая
RocCurveDisplay.from_predictions(y_test, y_lr_prob, ax=ax[1], name='Logistic Regression')
ax[1].set_title("ROC Curve")
ax[1].set_ylim(0, 1.05)

plt.tight_layout()

**Задание:** Постройте таблицу точности для набора данных wbdc. Сделайте по таблице метрик на обучающей и тестовой выборках. В таблице сравните разные преобразования признаков и гиперпараметры (регуляризацию). Можно сделать три-четыре эксперимента. 
- На каком эксперименте получилось достичь лучшего качества на трейне?
- А на тесте?
- Переобучается ли модель?

In [ ]:
target = 'diagnosis'
features = list(df.columns)
features.remove('diagnosis')

In [ ]:
X = df[features]
y = df[[target]]
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values.reshape(-1), train_size=0.8, shuffle=True)

_Четыре эксперимента: в них отличается сила регуляризации и метод масштабирования признаков (или вообще его отсутствие)._

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

experiments = [
    {"name": "No Scaling, C=1.0", "scaler": None, "C": 1.0},
    {"name": "StandardScaler, C=1.0", "scaler": StandardScaler(), "C": 1.0},
    {"name": "StandardScaler, C=0.01", "scaler": StandardScaler(), "C": 0.01},
    {"name": "MinMaxScaler, C=1.0", "scaler": MinMaxScaler(), "C": 1.0},
]

results = []

for exp in experiments:
    steps = []
    if exp["scaler"]:
        steps.append(("scaler", exp["scaler"]))
    steps.append(("clf", LogisticRegression(C=exp["C"], max_iter=1000)))
    
    pipe = Pipeline(steps)
    pipe.fit(X_train, y_train)
    
    for split, X, y in [("train", X_train, y_train), ("test", X_test, y_test)]:
        y_pred = pipe.predict(X)
        y_prob = pipe.predict_proba(X)[:, 1]
        
        results.append({
            "experiment": exp["name"],
            "split": split,
            "accuracy": accuracy_score(y, y_pred),
            "f1": f1_score(y, y_pred),
            "roc_auc": roc_auc_score(y, y_prob)
        })

df_results = pd.DataFrame(results)
df_results = df_results.pivot(index="experiment", columns="split", values=["accuracy", "f1", "roc_auc"])
df_results

_Если оценивать каечство по f1, то StandardScaler, C=1.0 показал себя лучше всего на train. То же самое и на test. Более сильные переобучения показали No Scaling, C=1.0 и StandardScaler, C=0.01, опять же в первом случае из-за отсутствия масштабирования, а во-втором - из-за слабой регуляризации._